<a href="https://colab.research.google.com/github/sheikmohdimran/Experiments_2021/blob/main/Vision/Effnet_TF2_Albumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow
from keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
import tensorflow_addons as tfa

In [ ]:
import pandas as pd
train_anno=pd.read_csv('/content/drive/MyDrive/Dataset/Annotations/Train Annotations.csv')

In [ ]:
files = train_data_dir.rglob('*.jpg')
input=pd.DataFrame(files,columns =['a'])
input['a']=input['a'].astype(str)
df_train=input['a'].str.split('/',expand=True)
train=input.join(df_train)[['a',7]]
train.columns=('file_path','label')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df,valid_df= train_test_split(train, test_size=0.20, random_state=42, stratify=train['label'])

In [ ]:
train_data_dir = Path('/content/drive/MyDrive/Dataset/Car Images/Train Images')
test_data_dir = Path('/content/drive/MyDrive/Dataset/Car Images/Test Images')

BATCH_SIZE = 32 
IMG_SIZE = 224

In [ ]:
!pip install -qqq tensorflow_addons
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor
!pip install -U albumentations

In [ ]:
import tensorflow as tf
from ImageDataAugmentor.image_data_augmentor import *
import albumentations

    
AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(p=0.5,limit=25),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=1.5, contrast_limit=0.1)
    ],p=1),
    #albumentations.HueSaturationValue(p=0.5),
    #albumentations.Affine(shear=20),
    albumentations.Cutout(num_holes=1, max_h_size=16,max_w_size = 16,p=1),
    #albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=15, p=.75),
    #albumentations.Blur(blur_limit=3),
    #albumentations.OpticalDistortion()
    #albumentations.RGBShift(p=0.5),
])

# dataloaders
train_datagen = ImageDataAugmentor(
        augment=AUGMENTATIONS,
        #validation_split=0.2,
        seed=42
        )

val_datagen = ImageDataAugmentor()#validation_split=0.2)

train_ds = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
        )

val_ds = val_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
        )

/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:690: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


Found 6515 validated image filenames belonging to 196 classes.
Found 1629 validated image filenames belonging to 196 classes.


In [ ]:
from tensorflow.keras import layers

In [ ]:
def build_model(num_classes=196,IMG_SIZE=224):
  inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
  model = tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, input_tensor=inputs, weights="imagenet")

  # Freeze the pretrained weights
  model.trainable = False

  # Rebuild top
  x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
  x = layers.BatchNormalization()(x)

  top_dropout_rate = 0.5
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

  # Compile
  return tf.keras.Model(inputs, outputs, name="EfficientNet")

In [ ]:
from functools import partial
INIT_LR=0.001
MAX_LR=0.1

steps_per_epoch = len(train_ds)
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
                                          maximal_learning_rate=MAX_LR,
                                          scale_fn=lambda x: 1/(2.**(x-1)),
                                          step_size=2 * steps_per_epoch
                                          )


optimizer = partial(SGD, momentum=0.9, nesterov=True)(clr)
optimizer = tfa.optimizers.SWA(optimizer)

model = build_model(num_classes=196,IMG_SIZE=224)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
epochs = 5 


hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1)

Epoch 1/5
204/204 [==============================] - 110s 498ms/step - loss: 7.0500 - accuracy: 0.0416 - val_loss: 4.0545 - val_accuracy: 0.1535
Epoch 2/5
204/204 [==============================] - 97s 476ms/step - loss: 6.5717 - accuracy: 0.0969 - val_loss: 3.5732 - val_accuracy: 0.1971
Epoch 3/5
204/204 [==============================] - 97s 476ms/step - loss: 4.4010 - accuracy: 0.1308 - val_loss: 2.9661 - val_accuracy: 0.2971
Epoch 4/5
204/204 [==============================] - 97s 476ms/step - loss: 3.8863 - accuracy: 0.1959 - val_loss: 2.6550 - val_accuracy: 0.3634
Epoch 5/5
204/204 [==============================] - 97s 476ms/step - loss: 3.7311 - accuracy: 0.2177 - val_loss: 2.6448 - val_accuracy: 0.3560


In [ ]:
checkpoint_path = "./cp-{iter:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
model.save_weights(checkpoint_path.format(iter=1))

In [ ]:
IMG_SIZE = 300

train_ds = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        #subset='training'
        )

val_ds = val_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        #subset='validation'
        )

model = build_model(num_classes=196,IMG_SIZE=IMG_SIZE)
model.load_weights('./cp-0001.ckpt')

optimizer = partial(SGD, momentum=0.9, nesterov=True)(clr)
optimizer = tfa.optimizers.SWA(optimizer)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1)

Found 6515 validated image filenames belonging to 196 classes.
Found 1629 validated image filenames belonging to 196 classes.
Epoch 1/5
204/204 [==============================] - 120s 549ms/step - loss: 3.6617 - accuracy: 0.2235 - val_loss: 2.5010 - val_accuracy: 0.3622
Epoch 2/5
204/204 [==============================] - 109s 532ms/step - loss: 3.5904 - accuracy: 0.2347 - val_loss: 2.2937 - val_accuracy: 0.4260
Epoch 3/5
204/204 [==============================] - 109s 534ms/step - loss: 3.3886 - accuracy: 0.2761 - val_loss: 2.1836 - val_accuracy: 0.4708
Epoch 4/5
204/204 [==============================] - 111s 543ms/step - loss: 3.2724 - accuracy: 0.2904 - val_loss: 2.1601 - val_accuracy: 0.4843
Epoch 5/5
204/204 [==============================] - 109s 534ms/step - loss: 3.2679 - accuracy: 0.2958 - val_loss: 2.1739 - val_accuracy: 0.4635


In [ ]:
model.save_weights(checkpoint_path.format(iter=2))

In [ ]:
for layer in model.layers[-20:]:
  if not isinstance(layer, layers.BatchNormalization):
    layer.trainable = True

INIT_LR=0.001
MAX_LR=0.1

steps_per_epoch = len(train_ds)
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
                                          maximal_learning_rate=MAX_LR,
                                          scale_fn=lambda x: 1/(2.**(x-1)),
                                          step_size=2 * steps_per_epoch
                                          )

model.load_weights('./cp-0002.ckpt')

model.compile(optimizer=tfa.optimizers.LAMB(), loss="categorical_crossentropy", metrics=["accuracy"])


epochs = 20  
model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1)

Epoch 1/20
204/204 [==============================] - 121s 550ms/step - loss: 3.2230 - accuracy: 0.3028 - val_loss: 1.9066 - val_accuracy: 0.5095
Epoch 2/20
204/204 [==============================] - 110s 539ms/step - loss: 3.0317 - accuracy: 0.3460 - val_loss: 1.7764 - val_accuracy: 0.5556
Epoch 3/20
204/204 [==============================] - 110s 540ms/step - loss: 2.8950 - accuracy: 0.3698 - val_loss: 1.6827 - val_accuracy: 0.5617
Epoch 4/20
204/204 [==============================] - 110s 539ms/step - loss: 2.7600 - accuracy: 0.3997 - val_loss: 1.5979 - val_accuracy: 0.6016
Epoch 5/20
204/204 [==============================] - 111s 543ms/step - loss: 2.7385 - accuracy: 0.4121 - val_loss: 1.5520 - val_accuracy: 0.5924
Epoch 6/20
204/204 [==============================] - 110s 539ms/step - loss: 2.6156 - accuracy: 0.4278 - val_loss: 1.4759 - val_accuracy: 0.6188
Epoch 7/20
204/204 [==============================] - 110s 538ms/step - loss: 2.5199 - accuracy: 0.4553 - val_loss: 1.4400 -

In [ ]:
model.save_weights(checkpoint_path.format(iter=3))

In [ ]:
epochs = 10
model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1)

Epoch 1/10
204/204 [==============================] - 110s 537ms/step - loss: 1.9979 - accuracy: 0.5827 - val_loss: 1.1859 - val_accuracy: 0.6826
Epoch 2/10
204/204 [==============================] - 110s 537ms/step - loss: 1.9366 - accuracy: 0.6011 - val_loss: 1.1717 - val_accuracy: 0.6888
Epoch 3/10
204/204 [==============================] - 110s 538ms/step - loss: 1.9642 - accuracy: 0.5913 - val_loss: 1.1697 - val_accuracy: 0.6906
Epoch 4/10
204/204 [==============================] - 110s 538ms/step - loss: 2.0110 - accuracy: 0.5863 - val_loss: 1.1819 - val_accuracy: 0.6863
Epoch 5/10
204/204 [==============================] - 110s 540ms/step - loss: 1.9438 - accuracy: 0.5997 - val_loss: 1.1565 - val_accuracy: 0.6894
Epoch 6/10
204/204 [==============================] - 110s 537ms/step - loss: 1.9288 - accuracy: 0.6020 - val_loss: 1.1372 - val_accuracy: 0.6980
Epoch 7/10
204/204 [==============================] - 110s 539ms/step - loss: 1.9087 - accuracy: 0.6072 - val_loss: 1.1367 -